In [1]:
!pip install transformers torch numpy

In [2]:
import torch
import numpy as np
from transformers import DistilBertTokenizer, DistilBertModel
import warnings
warnings.filterwarnings('ignore')

In [3]:
class TokenEmbeddingAnalyzer:

    def __init__(self, model_name='distilbert-base-uncased'):
        print(f"Loading {model_name} tokenizer and model...")
        self.tokenizer = DistilBertTokenizer.from_pretrained(model_name)
        self.model = DistilBertModel.from_pretrained(model_name)
        self.model.eval()
        print("Model loaded successfully!")

    def tokenize_and_analyze(self, sentences):
        all_tokens = []
        all_token_ids = []
        sentence_labels = []

        print("\n" + "=" * 80)
        print("TOKENIZATION ANALYSIS")
        print("=" * 80)

        total_tokens = 0

        for i, sentence in enumerate(sentences):
            tokens = self.tokenizer.tokenize(sentence)
            token_ids = self.tokenizer.convert_tokens_to_ids(tokens)

            all_tokens.extend(tokens)
            all_token_ids.extend(token_ids)
            sentence_labels.extend([i] * len(tokens))
            total_tokens += len(tokens)

            print(f"\n SENTENCE {i+1}: '{sentence}'")
            print(f"   Length: {len(sentence)} characters")
            print(f"   Tokens: {len(tokens)} tokens")
            print(f"   Token Details:")

            for j, (token, token_id) in enumerate(zip(tokens, token_ids)):
                display_token = token.replace('##', '##')
                print(f"      {j+1:2d}. '{display_token:15}' -> ID: {token_id:5d}")

        print(f"\n TOKENIZATION SUMMARY:")
        print(f"   Total sentences processed: {len(sentences)}")
        print(f"   Total tokens generated: {total_tokens}")
        print(f"   Average tokens per sentence: {total_tokens/len(sentences):.2f}")
        print(f"   Vocabulary size used: {len(set(all_token_ids))}")

        return {
            'tokens': all_tokens,
            'token_ids': all_token_ids,
            'sentence_labels': sentence_labels,
            'sentences': sentences
        }

    def extract_embeddings(self, sentences, show_details=True):
        if show_details:
            print("\n" + "=" * 80)
            print("EMBEDDING EXTRACTION")
            print("=" * 80)

        all_embeddings = []
        token_data = self.tokenize_and_analyze(sentences)

        for i, sentence in enumerate(sentences):
            if show_details:
                print(f"\n Processing sentence {i+1} for embeddings...")

            inputs = self.tokenizer(sentence, return_tensors='pt', padding=True, truncation=True)

            with torch.no_grad():
                outputs = self.model(**inputs)
                embeddings = outputs.last_hidden_state.squeeze(0)
                embeddings = embeddings[1:-1]

                all_embeddings.append(embeddings.numpy())

                if show_details:
                    print(f" Extracted {embeddings.shape[0]} token embeddings")
                    print(f" Embedding dimension: {embeddings.shape[1]}")

        final_embeddings = np.vstack(all_embeddings)

        if show_details:
            print(f"\n EMBEDDING SUMMARY:")
            print(f"   Total token embeddings: {final_embeddings.shape[0]}")
            print(f"   Embedding dimensions: {final_embeddings.shape[1]}")
            print(f"   Total parameters represented: {final_embeddings.size:,}")
            print(f"   Memory usage: {final_embeddings.nbytes / (1024*1024):.2f} MB")

        return final_embeddings, token_data

    def analyze_embeddings(self, embeddings, token_data):
        print(f"\n" + "=" * 80)
        print("EMBEDDING STATISTICAL ANALYSIS")
        print("=" * 80)

        print(f"\n Basic Statistics:")
        print(f"   Mean activation: {np.mean(embeddings):.6f}")
        print(f"   Standard deviation: {np.std(embeddings):.6f}")
        print(f"   Min value: {np.min(embeddings):.6f}")
        print(f"   Max value: {np.max(embeddings):.6f}")

        token_norms = np.linalg.norm(embeddings, axis=1)
        print(f"\n Per-token Analysis:")
        print(f"   Average token embedding norm: {np.mean(token_norms):.6f}")
        print(f"   Token norm std dev: {np.std(token_norms):.6f}")

        print(f"\n Token Similarity Analysis:")
        if len(embeddings) > 1:
            normalized_embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
            similarity_matrix = np.dot(normalized_embeddings, normalized_embeddings.T)

            np.fill_diagonal(similarity_matrix, -1)
            max_similarity_idx = np.unravel_index(np.argmax(similarity_matrix), similarity_matrix.shape)
            max_similarity = similarity_matrix[max_similarity_idx]

            token1 = token_data['tokens'][max_similarity_idx[0]]
            token2 = token_data['tokens'][max_similarity_idx[1]]

            print(f"   Most similar tokens: '{token1}' ↔ '{token2}'")
            print(f"   Cosine similarity: {max_similarity:.6f}")

In [4]:
def get_user_input():
    print("\n" + "=" * 80)
    print("INTERACTIVE TOKENIZER & EMBEDDING ANALYZER")
    print("=" * 80)
    print("\n Enter your sentences below:")
    print("   • Press Enter after each sentence")
    print("   • Press Enter on an empty line to finish")
    print("   • Enter 'quit' to exit the program")
    print("\n" + "-" * 50)

    sentences = []
    sentence_count = 1

    while True:
        try:
            sentence = input(f"Sentence {sentence_count}: ").strip()

            if sentence.lower() == 'quit':
                print("Goodbye!")
                return None

            if sentence == "":
                if sentences:
                    break
                else:
                    print("Please enter at least one sentence.")
                    continue

            sentences.append(sentence)
            sentence_count += 1

        except KeyboardInterrupt:
            print("\n\n Program interrupted by user. Goodbye!")
            return None
        except EOFError:
            print("\n\n Input stream ended. Goodbye!")
            return None

    print(f"\n Collected {len(sentences)} sentence(s) for analysis.")
    return sentences

In [5]:
def main():
    print(" DISTILBERT TOKENIZER + EMBEDDING ANALYZER")
    print("=" * 80)

    try:
        analyzer = TokenEmbeddingAnalyzer()
    except Exception as e:
        print(f" Error loading model: {e}")
        return

    while True:
        sentences = get_user_input()

        if sentences is None:
            break

        try:
            embeddings, token_data = analyzer.extract_embeddings(sentences, show_details=True)

            analyzer.analyze_embeddings(embeddings, token_data)

            print(f"\n" + "=" * 80)
            continue_choice = input(" Would you like to analyze more sentences? (y/n): ").strip().lower()

            if continue_choice not in ['y', 'yes']:
                print(" Thank you for using the analyzer!")
                break

        except Exception as e:
            print(f" Error during analysis: {e}")
            print(" Please try again with different input.")

In [6]:
if __name__ == "__main__":
    main()

 DISTILBERT TOKENIZER + EMBEDDING ANALYZER
Loading distilbert-base-uncased tokenizer and model...


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Model loaded successfully!

INTERACTIVE TOKENIZER & EMBEDDING ANALYZER

 Enter your sentences below:
   • Press Enter after each sentence
   • Press Enter on an empty line to finish
   • Enter 'quit' to exit the program

--------------------------------------------------
Sentence 1: I have started with summer training projects.
Sentence 2: In that I have three projects.
Sentence 3: The projects are focused on LLMs.
Sentence 4: Since, I am working on LLMs, it will help me a lot.
Sentence 5: It will help me to base up my skills.
Sentence 6: That is much needed.
Sentence 7: I am focusing to add these projects in my CV.
Sentence 8: 

 Collected 7 sentence(s) for analysis.

EMBEDDING EXTRACTION

TOKENIZATION ANALYSIS

 SENTENCE 1: 'I have started with summer training projects.'
   Length: 45 characters
   Tokens: 8 tokens
   Token Details:
       1. 'i              ' -> ID:  1045
       2. 'have           ' -> ID:  2031
       3. 'started        ' -> ID:  2318
       4. 'with           ' ->